In [1]:
! nvidia-smi
! pip install optuna xgboost==1.6.1 pytorch-tabnet

Sun Jan  1 09:03:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pandas as pd
import numpy as np
# from xgboost import XGBRegressor
import torch
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.metrics import accuracy_score,mean_squared_error,mean_absolute_error,r2_score
import optuna as opt
import joblib
import random
import os

In [3]:
def make_save_cv_model(i,model_name,model,optim,mse_loss,trial_data,output_path="./"):

    ''' This function saves cross validation model in the corresponding directory ( if the path does not exist it creates the path for it'''


    if os.path.exists(os.path.join(output_path,f"trial_{i}_{model_name}_{optim}")):
        joblib.dump(model, os.path.join(output_path,f"trial_{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"trial_{i}_{model_name}_{optim}/mse_loss.txt"),"w+") as file:file.write(f"{mse_loss}")
        joblib.dump(trial_data, os.path.join(output_path,f"trial_{i}_{model_name}_{optim}/{i}_trial_data.z"))
    else:
        os.mkdir(os.path.join(output_path,f"trial_{i}_{model_name}_{optim}"))
        joblib.dump(model, os.path.join(output_path,f"trial_{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"trial_{i}_{model_name}_{optim}/mse_loss.txt"),"w+") as file:file.write(f"{mse_loss}")
        joblib.dump(trial_data, os.path.join(output_path,f"trial_{i}_{model_name}_{optim}/{i}_trial_data.z"))

In [4]:
def save_cv_model(i,model_name,model,optim,mse_loss,output_path="./"):

    ''' This function saves cross validation model in the corresponding directory ( if the path does not exist it creates the path for it'''


    if os.path.exists(os.path.join(output_path,f"{i}_{model_name}_{optim}")):
        joblib.dump(model, os.path.join(output_path,f"{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"{i}_{model_name}_{optim}/losses_{fold}.txt"),"w+") as file:file.write(f"{mse_loss}")
    else:
        os.mkdir(os.path.join(output_path,f"{i}_{model_name}_{optim}"))
        joblib.dump(model, os.path.join(output_path,f"{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"{i}_{model_name}_{optim}/losses_{fold}.txt"),"w+") as file:file.write(f"{mse_loss}")

In [5]:
def get_trial_data(trial) -> list:
  ''' This function takes the trial objects and returns the dictionary containing the trial details for plotting and comparing purposes '''
  trial_data = trial.get_trials()
  value_dict = {}
  for i in trial_data:
    print(i.params)
    value_dict[i.number] = {"params": i.params , "rmse": i.values}
    print(f"{i.number} : {i.values}")
  return value_dict

In [6]:
def train_trial(fold_dict,fold,model_name,sc_df,tar_col,optim,optim_trial,k_folds,tar_cols="",verbose=1):

    ''' this function is used to train the model with parameters optimization using optuna and cross validation using stratified k_folds'''

    y = sc_df[tar_col]
    print(y.shape)
    x = sc_df.drop([tar_col],axis=1)
    print(x.shape)
    model_name = model_name 
    def objective(trial):
      train_index = fold_dict[fold]["train"]
      test_index = fold_dict[fold]["test"]
      clf = TabNetRegressor(n_d=trial.suggest_int("n_d", 8, 64),
                                    n_a =trial.suggest_int("n_a", 8, 64),
                                    n_steps = trial.suggest_int("n_steps",3,10),
                                    gamma =trial.suggest_float("gamma", 1.0, 2.0),
                                    n_independent = trial.suggest_int("n_independent",1,5),
                                    n_shared = trial.suggest_int("n_shared",1,5),
                                    momentum = trial.suggest_float("momentum", 0.01, 0.4),
                                    optimizer_fn = torch.optim.Adam,
                                    # scheduler_fn = torch.optim.lr_scheduler,
                                    # scheduler_params = {"gamma" :trial.suggest_float("sch-gamma", 0.5, 0.95), "step_size": trial.suggest_int("sch_step_size", 10, 20, 2)},
                                    verbose = verbose,
                                    device_name = "cuda" if torch.cuda.is_available() else "cpu"
                                    )
      # print(f" train_index :: {train_index}")
      # print(f" test_index :: {test_index}")
      X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
      # print(X_train.shape, X_test.shape)
      X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
      Y_train, Y_test = y.iloc[train_index].to_numpy(dtype=np.float64), y.iloc[test_index].to_numpy(np.float64)
      # Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
      print(X_train.shape)
      print(Y_train.shape)
      print(X_test.shape)
      print(Y_test.shape)
      Y_train = Y_train.reshape(-1,1)
      clf.fit(X_train, Y_train)
      Y_pred = clf.predict(X_test)
      mse_error = mean_squared_error(Y_pred, Y_test, squared=False) 
      return mse_error

    print(f"Starting optimization for fold : [{fold}/{k_folds}]")
    study = opt.create_study(direction='minimize')
    study.optimize(objective, n_trials=optim_trial)
    best_params = study.best_params
    trial_data = get_trial_data(study)
    print(f" Best params for fold : [{fold}/{k_folds}]")
    print(best_params)
    train_index = fold_dict[fold]["train"]
    test_index = fold_dict[fold]["test"]
    X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
    # print(X_train.shape, X_test.shape)
    X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
    Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
    Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
    clf_model = TabNetRegressor(**study.best_params,device_name= "cuda" if torch.cuda.is_available() else "cpu")
    Y_train = Y_train.reshape(-1,1)
    clf_model.fit(X_train,Y_train)
    Y_pred = clf_model.predict(X_test)
    error = {"mse_error" : mean_squared_error(Y_pred, Y_test, squared=False),
    "mae_error" : mean_absolute_error(Y_pred,Y_test),
    "rmse_error" : mean_squared_error(Y_pred, Y_test),
    "r2_score" : r2_score(Y_pred,Y_test) }
    # try:
    print("[++] Saving the model and parameters in corresponding directories")
    make_save_cv_model(fold,model_name,clf_model,optim,mse_loss=error,trial_data=trial_data)
    return trial_data,best_params

In [7]:
def train(fold_dict,fold,model_name,sc_df,tar_col,optim,k_folds,best_params,tar_cols="",verbose=1):

    ''' this function is used to train the model with parameters optimization using optuna and cross validation using stratified k_folds'''

    y = sc_df[tar_col]
    print(y.shape)
    x = sc_df.drop([tar_col],axis=1)
    print(x.shape)
    model_name = model_name 
    train_index = fold_dict[fold]["train"]
    test_index = fold_dict[fold]["test"]
    X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
    # print(X_train.shape, X_test.shape)
    X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
    Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
    Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
    clf_model = TabNetRegressor(**best_params,device_name= "cuda" if torch.cuda.is_available() else "cpu")
    Y_train = Y_train.reshape(-1,1)
    clf_model.fit(X_train,Y_train)
    Y_pred = clf_model.predict(X_test)
    error = {
    "mse_error" : mean_squared_error(Y_pred, Y_test, squared=False),
    "mae_error" : mean_absolute_error(Y_pred,Y_test),
    "rmse_error" : mean_squared_error(Y_pred, Y_test),
    "r2_score" : r2_score(Y_pred,Y_test) }
    # try:
    print("[++] Saving the model and parameters in corresponding directories")
    save_cv_model(fold,model_name,clf_model,optim,mse_loss=error)

In [8]:
use_df = pd.read_csv("../input/perov-scaled-data/scaled_trainable.csv")
tar_col = "JV_default_PCE_numeric"
model_name = "tabnet_reg"
fold_dict = joblib.load("../input/perov-fold-data/fold_data_export.z")
optim = "adam"
k_folds = 20
trial_fold = random.choice([x for x in range(k_folds)])
num_trials = 15

trial_data,best_params = train_trial(fold_dict = fold_dict,
      fold = trial_fold,
      k_folds=k_folds,
      model_name=model_name,
      sc_df=use_df,
      tar_col=tar_col,
      optim = optim,
      optim_trial = num_trials)
for key,value in trial_data.items():
  print(f"{key}: {value['rmse']}")
print(f"[++] Ended the training process for fold {trial_fold}")


main_folds = [x for x in range(k_folds)]
for fold in main_folds:
    train(fold_dict = fold_dict,
          fold = fold,
          k_folds=k_folds,
          model_name=model_name,
          sc_df=use_df,
          tar_col=tar_col,
          optim = optim,
          best_params = best_params)
    print(f"[++] Ended the training process for fold {fold}")

[I 2023-01-01 09:04:02,309] A new study created in memory with name: no-name-76564aa2-c7c1-48d9-9e7e-d0eb358b4eb4


(46820,)
(46820, 103)
Starting optimization for fold : [0/20]
(44479, 103)
(44479,)
(2341, 103)
(2341,)


/opt/conda/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/opt/conda/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 216.60689|  0:00:04s
epoch 1  | loss: 47.67343|  0:00:07s
epoch 2  | loss: 39.05951|  0:00:11s
epoch 3  | loss: 37.10037|  0:00:14s
epoch 4  | loss: 33.52174|  0:00:18s
epoch 5  | loss: 26.51078|  0:00:21s
epoch 6  | loss: 22.62607|  0:00:24s
epoch 7  | loss: 30.99096|  0:00:27s
epoch 8  | loss: 23.31654|  0:00:31s
epoch 9  | loss: 19.98966|  0:00:34s
epoch 10 | loss: 18.22681|  0:00:37s
epoch 11 | loss: 16.53621|  0:00:41s
epoch 12 | loss: 16.12404|  0:00:44s
epoch 13 | loss: 15.93442|  0:00:48s
epoch 14 | loss: 16.92738|  0:00:51s
epoch 15 | loss: 16.50148|  0:00:54s
epoch 16 | loss: 15.27087|  0:00:57s
epoch 17 | loss: 15.48071|  0:01:01s
epoch 18 | loss: 14.67863|  0:01:04s
epoch 19 | loss: 14.2742 |  0:01:08s
epoch 20 | loss: 13.49315|  0:01:12s
epoch 21 | loss: 12.84221|  0:01:15s
epoch 22 | loss: 12.57966|  0:01:18s
epoch 23 | loss: 12.63678|  0:01:21s
epoch 24 | loss: 12.08361|  0:01:25s
epoch 25 | loss: 11.8734 |  0:01:28s
epoch 26 | loss: 11.84081|  0:01:31s


[I 2023-01-01 09:09:48,992] Trial 0 finished with value: 2.869403157472206 and parameters: {'n_d': 16, 'n_a': 17, 'n_steps': 8, 'gamma': 1.1792901351310356, 'n_independent': 1, 'n_shared': 3, 'momentum': 0.13446266890938988}. Best is trial 0 with value: 2.869403157472206.
/opt/conda/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/opt/conda/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


...

epoch 0  | loss: 148.23604|  0:00:03s
epoch 1  | loss: 29.31938|  0:00:06s
epoch 2  | loss: 23.19788|  0:00:08s
epoch 3  | loss: 19.2165 |  0:00:11s
epoch 4  | loss: 16.18241|  0:00:14s
epoch 5  | loss: 15.10435|  0:00:17s
epoch 6  | loss: 13.94649|  0:00:20s
epoch 7  | loss: 12.84248|  0:00:22s
epoch 8  | loss: 11.89803|  0:00:25s
epoch 9  | loss: 11.20357|  0:00:28s
epoch 10 | loss: 11.37096|  0:00:31s
epoch 11 | loss: 11.15201|  0:00:33s
epoch 12 | loss: 10.47358|  0:00:37s
epoch 13 | loss: 10.47713|  0:00:40s
epoch 14 | loss: 10.03987|  0:00:42s
epoch 15 | loss: 9.99763 |  0:00:45s
epoch 16 | loss: 9.6724  |  0:00:48s
epoch 17 | loss: 9.47944 |  0:00:51s
epoch 18 | loss: 9.49178 |  0:00:54s
epoch 19 | loss: 9.2816  |  0:00:57s
epoch 20 | loss: 9.08086 |  0:00:59s
epoch 21 | loss: 9.10183 |  0:01:02s
epoch 22 | loss: 8.98857 |  0:01:05s
epoch 23 | loss: 8.87135 |  0:01:08s
epoch 24 | loss: 8.77124 |  0:01:11s
epoch 25 | loss: 8.69698 |  0:01:14s
epoch 26 | loss: 8.49805 |  0:01:16s


/opt/conda/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/opt/conda/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 148.63062|  0:00:03s
epoch 1  | loss: 28.95937|  0:00:06s
epoch 2  | loss: 23.68539|  0:00:08s
epoch 3  | loss: 23.68478|  0:00:11s
epoch 4  | loss: 19.04347|  0:00:14s
epoch 5  | loss: 17.7189 |  0:00:17s
epoch 6  | loss: 18.51848|  0:00:19s
epoch 7  | loss: 19.0353 |  0:00:22s
epoch 8  | loss: 14.97094|  0:00:25s
epoch 9  | loss: 13.22594|  0:00:28s
epoch 10 | loss: 12.76025|  0:00:31s
epoch 11 | loss: 12.36225|  0:00:34s
epoch 12 | loss: 12.03362|  0:00:37s
epoch 13 | loss: 11.61599|  0:00:39s
epoch 14 | loss: 11.19869|  0:00:42s
epoch 15 | loss: 10.91836|  0:00:45s
epoch 16 | loss: 10.44852|  0:00:47s
epoch 17 | loss: 10.25653|  0:00:50s
epoch 18 | loss: 10.20189|  0:00:53s
epoch 19 | loss: 9.90026 |  0:00:56s
epoch 20 | loss: 9.69548 |  0:00:59s
epoch 21 | loss: 9.73738 |  0:01:01s
epoch 22 | loss: 9.50868 |  0:01:05s
epoch 23 | loss: 9.28002 |  0:01:07s
epoch 24 | loss: 10.10354|  0:01:10s
epoch 25 | loss: 9.42442 |  0:01:13s
epoch 26 | loss: 9.18066 |  0:01:16s


/opt/conda/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/opt/conda/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 154.33243|  0:00:02s
epoch 1  | loss: 29.6186 |  0:00:05s
epoch 2  | loss: 24.68119|  0:00:08s
epoch 3  | loss: 21.48843|  0:00:11s
epoch 4  | loss: 19.19708|  0:00:14s
epoch 5  | loss: 16.72328|  0:00:16s
epoch 6  | loss: 15.49621|  0:00:19s
epoch 7  | loss: 15.7777 |  0:00:22s
epoch 8  | loss: 14.11733|  0:00:25s
epoch 9  | loss: 13.34834|  0:00:28s
epoch 10 | loss: 12.74193|  0:00:31s
epoch 11 | loss: 12.81393|  0:00:33s
epoch 12 | loss: 11.98686|  0:00:36s
epoch 13 | loss: 12.02584|  0:00:39s
epoch 14 | loss: 11.65167|  0:00:42s
epoch 15 | loss: 11.41206|  0:00:44s
epoch 16 | loss: 17.50116|  0:00:47s
epoch 17 | loss: 11.16322|  0:00:50s
epoch 18 | loss: 10.66988|  0:00:53s
epoch 19 | loss: 10.81375|  0:00:55s
epoch 20 | loss: 10.61685|  0:00:58s
epoch 21 | loss: 10.40517|  0:01:01s
epoch 22 | loss: 10.24871|  0:01:04s
epoch 23 | loss: 10.06812|  0:01:07s
epoch 24 | loss: 9.99337 |  0:01:10s
epoch 25 | loss: 9.90096 |  0:01:13s
epoch 26 | loss: 9.64398 |  0:01:15s
